# **GALAXY MORPHOLOGY CLASSIFICATION USING DEEP LEARNING AND EXPLAINABLE AI**

## **Import Libraries and Initial Loading**

In [1]:
!pip install imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 4.0 MB/s eta 0:00:00


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import tensorflow as tf
from google.colab import drive
import cv2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras import layers, models, optimizers, regularizers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, models


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os

# Path to your Google Drive folder
zip_file_path_1 = '/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge v2/images_training_rev1.zip'
zip_file_path_2 = '/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge v2/training_solutions_rev1.zip'

# Folder where you want to extract the contents
extract_folder_1 = '/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge v2/images_training_rev1'
extract_folder_2 = '/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge v2/training_solutions_rev1'

# Unzipping the files
with zipfile.ZipFile(zip_file_path_1, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_1)

with zipfile.ZipFile(zip_file_path_2, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_2)

In [4]:

train_images_path = "/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge v2/images_training_rev1/images_training_rev1"
labels_path = "/content/drive/MyDrive/galaxy-zoo-the-galaxy-challenge v2/training_solutions_rev1/training_solutions_rev1.csv"



## **Preprocessing**

### **Labels**

In [5]:
# Load labels
labels_df = pd.read_csv(labels_path)


In [6]:
labels_df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [7]:
labels_df.shape

(61578, 38)

In [8]:
labels_df['image_path'] = labels_df['GalaxyID'].astype(str) + '.jpg'
labels_df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image_path
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512,100008.jpg
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000,100023.jpg
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100053.jpg
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000,100078.jpg
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100090.jpg


In [9]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61578 entries, 0 to 61577
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   GalaxyID    61578 non-null  int64  
 1   Class1.1    61578 non-null  float64
 2   Class1.2    61578 non-null  float64
 3   Class1.3    61578 non-null  float64
 4   Class2.1    61578 non-null  float64
 5   Class2.2    61578 non-null  float64
 6   Class3.1    61578 non-null  float64
 7   Class3.2    61578 non-null  float64
 8   Class4.1    61578 non-null  float64
 9   Class4.2    61578 non-null  float64
 10  Class5.1    61578 non-null  float64
 11  Class5.2    61578 non-null  float64
 12  Class5.3    61578 non-null  float64
 13  Class5.4    61578 non-null  float64
 14  Class6.1    61578 non-null  float64
 15  Class6.2    61578 non-null  float64
 16  Class7.1    61578 non-null  float64
 17  Class7.2    61578 non-null  float64
 18  Class7.3    61578 non-null  float64
 19  Class8.1    61578 non-nul

In [10]:
# missing values
missing_values = labels_df.isnull().sum()

print("Missing Values in Each Column:")
print(missing_values)

if missing_values.sum() > 0:
    print("\nThere are missing values in the dataset.")
else:
    print("\nNo missing values in the dataset.")

Missing Values in Each Column:
GalaxyID      0
Class1.1      0
Class1.2      0
Class1.3      0
Class2.1      0
Class2.2      0
Class3.1      0
Class3.2      0
Class4.1      0
Class4.2      0
Class5.1      0
Class5.2      0
Class5.3      0
Class5.4      0
Class6.1      0
Class6.2      0
Class7.1      0
Class7.2      0
Class7.3      0
Class8.1      0
Class8.2      0
Class8.3      0
Class8.4      0
Class8.5      0
Class8.6      0
Class8.7      0
Class9.1      0
Class9.2      0
Class9.3      0
Class10.1     0
Class10.2     0
Class10.3     0
Class11.1     0
Class11.2     0
Class11.3     0
Class11.4     0
Class11.5     0
Class11.6     0
image_path    0
dtype: int64

No missing values in the dataset.


### **Images**

In [12]:
# number of images
train_image_count = len(os.listdir(train_images_path))

print(f"Number of images in training folder: {train_image_count}")

Number of images in training folder: 61578


In [14]:
# list of image file names in the training folder
training_images = set(os.listdir(train_images_path))

filtered_labels_df = labels_df[labels_df['image_path'].isin(training_images)].copy()

filtered_labels_df.reset_index(drop=True, inplace=True)

print(f"Number of images in training folder: {len(training_images)}")
print(f"Number of rows in filtered_labels_df: {filtered_labels_df.shape[0]}")
filtered_labels_df.head()

Number of images in training folder: 61578
Number of rows in filtered_labels_df: 61578


,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image_path
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512,100008.jpg
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000,100023.jpg
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100053.jpg
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000,100078.jpg
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100090.jpg


In [24]:
def classify_galaxy(row):
    if row['Class1.2'] >= 0.8 and row['Class4.1'] >= 0.8:
        return 'Spiral'
    elif row['Class1.1'] >= 0.8:
        return 'Elliptical'
    elif row['Class1.2'] >= 0.6 and row['Class4.2'] >= 0.6:
        return 'Lenticular'
    elif row['Class8.4'] >= 0.6:
        return 'Irregular'
    elif row['Class6.1'] >= 0.6 or row['Class8.1'] >= 0.6 or row['Class8.2'] >= 0.6 or \
         row['Class8.3'] >= 0.6 or row['Class8.5'] >= 0.6 or row['Class8.6'] >= 0.6 or row['Class8.7'] >= 0.8:
        return 'Peculiar'
    # New galaxy types
    elif row['Class1.1'] >= 0.6 and row['Class4.1'] >= 0.6:
        return 'Mixed Type'  # Galaxies with mixed features
    elif row['Class1.2'] >= 0.5 and row['Class4.2'] >= 0.5:
        return 'Transitional'  # Galaxies in transition
    elif row['Class8.5'] >= 0.5 or row['Class8.6'] >= 0.5:
        return 'Hybrid'  # Hybrid galaxies with mixed features
    elif row['Class8.3'] >= 0.5:
        return 'Fragmented'  # Galaxies with fragmented or disturbed features
    else:
        return 'Unclear'  # For galaxies that are hard to classify but not completely uncertain

# Apply the classification
filtered_labels_df['Galaxy_Type'] = filtered_labels_df.apply(classify_galaxy, axis=1)

# Check the distribution of galaxy types again
print(filtered_labels_df['Galaxy_Type'].value_counts())


Galaxy_Type
Unclear         39540
Elliptical       8132
Spiral           4793
Peculiar         4406
Transitional     2483
Lenticular       2135
Irregular          76
Hybrid              7
Fragmented          6
Name: count, dtype: int64


In [17]:
# --- Check Class Distribution and Take Top 5 Classes ---
# Check the distribution of the classes
class_counts = filtered_labels_df["class"].value_counts()

# Show the top 5 classes
top_5_classes = class_counts.head(5)
print("Top 5 classes by frequency:")
print(top_5_classes)

# Define the top 5 classes (including 'uncertain')
top_classes = top_5_classes.index.tolist()

# Create a new DataFrame with galaxy_id and class, filtered by the top 5 classes
filtered_labels_df1 = filtered_labels_df[filtered_labels_df["class"].isin(top_classes)][["GalaxyID", "class","image_path"]]

# Check the new DataFrame
print("Filtered DataFrame with Galaxy ID and Class for Top 5 Classes:")
print(filtered_labels_df.head())
print(filtered_labels_df.shape)

# Optionally, you can save the new DataFrame to a file, like a CSV
# filtered_labels_df.to_csv('filtered_top_5_classes.csv', index=False)


Top 5 classes by frequency:
class
uncertain       11503
spiral_arm       2917
smooth           2283
disk_edge_on      511
bar_feature       285
Name: count, dtype: int64
Filtered DataFrame with Galaxy ID and Class for Top 5 Classes:
   GalaxyID  Class1.1  Class1.2  Class1.3  Class2.1  Class2.2  Class3.1  \
0    100008  0.383147  0.616853  0.000000  0.000000  0.616853  0.038452   
1    100023  0.327001  0.663777  0.009222  0.031178  0.632599  0.467370   
2    100053  0.765717  0.177352  0.056931  0.000000  0.177352  0.000000   
3    100078  0.693377  0.238564  0.068059  0.000000  0.238564  0.109493   
4    100090  0.933839  0.000000  0.066161  0.000000  0.000000  0.000000   

   Class3.2  Class4.1  Class4.2  ...  Class11.6  image_path       class  \
0  0.578401  0.418398  0.198455  ...   0.325512  100008.jpg   uncertain   
1  0.165229  0.591328  0.041271  ...   0.000000  100023.jpg  spiral_arm   
2  0.177352  0.000000  0.177352  ...   0.000000  100053.jpg   uncertain   
3  0.129071  0.1

In [18]:
filtered_labels_df1.head()

,GalaxyID,class,image_path
0,100008,uncertain,100008.jpg
1,100023,spiral_arm,100023.jpg
2,100053,uncertain,100053.jpg
3,100078,uncertain,100078.jpg
4,100090,smooth,100090.jpg


In [19]:
# Define the target number of samples per class
target_samples = 500

# Create an empty DataFrame to store the balanced dataset
balanced_data = pd.DataFrame(columns=["GalaxyID", "class"])

# Loop through each class and sample the required number of images
for class_label in ["uncertain", "spiral_arm", "smooth", "disk_edge_on", "bar_feature"]:
    class_data = filtered_labels_df[filtered_labels_df["class"] == class_label][["GalaxyID", "class","image_path"]]
    if len(class_data) >= target_samples:
        # Sample exactly 500 images if possible
        sampled_data = class_data.sample(n=target_samples, random_state=42)
    else:
        # Take all available samples if less than 500
        sampled_data = class_data
        print(f"Warning: Class '{class_label}' has only {len(class_data)} samples. Using all available.")

    # Append the sampled data to the balanced DataFrame
    balanced_data = pd.concat([balanced_data, sampled_data], ignore_index=True)

# Reset the index of the balanced DataFrame
balanced_data.reset_index(drop=True, inplace=True)

# Check the structure of the new balanced DataFrame
print("Balanced DataFrame:")
print(balanced_data.head())

# Check the distribution in the new balanced DataFrame
print("\nClass distribution in the balanced dataset:")
print(balanced_data["class"].value_counts())


Balanced DataFrame:
  GalaxyID      class  image_path
0   337092  uncertain  337092.jpg
1   243130  uncertain  243130.jpg
2   228215  uncertain  228215.jpg
3   210378  uncertain  210378.jpg
4   251453  uncertain  251453.jpg

Class distribution in the balanced dataset:
class
uncertain       500
spiral_arm      500
smooth          500
disk_edge_on    500
bar_feature     285
Name: count, dtype: int64


In [20]:
# Apply Label Encoding to the class column in balanced_data
label_encoder = LabelEncoder()
balanced_data['class_encoded'] = label_encoder.fit_transform(balanced_data['class'])

# Check the mapping of labels to encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Encoding Mapping:")
print(label_mapping)

# Check the updated balanced_data DataFrame
print("Balanced DataFrame with Encoded Classes:")
print(balanced_data.head())

Label Encoding Mapping:
{'bar_feature': 0, 'disk_edge_on': 1, 'smooth': 2, 'spiral_arm': 3, 'uncertain': 4}
Balanced DataFrame with Encoded Classes:
  GalaxyID      class  image_path  class_encoded
0   337092  uncertain  337092.jpg              4
1   243130  uncertain  243130.jpg              4
2   228215  uncertain  228215.jpg              4
3   210378  uncertain  210378.jpg              4
4   251453  uncertain  251453.jpg              4


In [21]:
import os
from PIL import Image


# Iterate through the GalaxyIDs in the balanced data
for galaxy_id in balanced_data["GalaxyID"].head():  # Adjust `.head()` or remove to check all
    image_file = f"{galaxy_id}.jpg"  # Assuming the images are in .jpeg format
    image_path = os.path.join(train_images_path, image_file)
    if os.path.exists(image_path):
        with Image.open(image_path) as img:
            print(f"GalaxyID: {galaxy_id}, Image: {image_file}, Size: {img.size}")
    else:
        print(f"Image for GalaxyID: {galaxy_id} not found.")


GalaxyID: 337092, Image: 337092.jpg, Size: (424, 424)
GalaxyID: 243130, Image: 243130.jpg, Size: (424, 424)
GalaxyID: 228215, Image: 228215.jpg, Size: (424, 424)
GalaxyID: 210378, Image: 210378.jpg, Size: (424, 424)
GalaxyID: 251453, Image: 251453.jpg, Size: (424, 424)


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    return img_array

# Create full paths for image files
image_paths = balanced_data['image_path'].apply(lambda x: os.path.join(train_images_path, x)).tolist()

# Preprocess the images
image_data = np.array([preprocess_image(path) for path in image_paths])
print(f"Image data shape: {image_data.shape}")

# Save preprocessed data
np.save(os.path.join(Data_path, "preprocessed_image_data.npy"), image_data)
print("Preprocessed data saved successfully!")

KeyboardInterrupt: 

In [22]:
# Load saved preprocessed data (optional, for reusability)
image_data_file = os.path.join(Data_path, "preprocessed_image_data.npy")
image_data = np.load(image_data_file)

## **EDA**

## **Data Splitting**

In [23]:
# Parameters for splitting
test_size = 0.2
val_size = 0.2
random_state = 42

# Use encoded labels for splitting
labels = balanced_data['class_encoded'].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    image_data, labels, test_size=test_size, random_state=random_state
)

print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}")
print(f"Testing labels shape: {y_test.shape}")

# Further split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=random_state
)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")

Training data shape: (1828, 224, 224, 3)
Training labels shape: (1828,)
Testing data shape: (457, 224, 224, 3)
Testing labels shape: (457,)
Training data shape: (1462, 224, 224, 3)
Validation data shape: (366, 224, 224, 3)
Training labels shape: (1462,)
Validation labels shape: (366,)


## **Augmentation**

In [24]:
# Get indices of the DataFrame
indices = balanced_data.index

# Split indices for training and testing
train_indices, test_indices = train_test_split(
    indices, test_size=test_size, random_state=random_state
)

# Further split training indices into training and validation sets
train_indices, val_indices = train_test_split(
    train_indices, test_size=val_size, random_state=random_state
)

# Create new DataFrames for each subset using the indices
train_df = balanced_data.iloc[train_indices]
val_df = balanced_data.iloc[val_indices]
test_df = balanced_data.iloc[test_indices]

# Data Augmentation for Training and Rescaling for Validation and Testing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale images to [0,1] range
    rotation_range=40,  # Random rotations
    width_shift_range=0.2,  # Horizontal shifts
    height_shift_range=0.2,  # Vertical shifts
    shear_range=0.2,  # Random shear transformations
    zoom_range=0.2,  # Zoom in/out
    horizontal_flip=True,  # Random horizontal flips
    fill_mode='nearest'  # Fill in any pixels that are created after transformations
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train Generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_images_path,
    x_col='image_path',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Validation Generator
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_images_path,
    x_col='image_path',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Test Generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=train_images_path,
    x_col='image_path',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Print shapes to ensure data is correctly split
print(f"Training data shape: {train_generator.samples}")
print(f"Validation data shape: {val_generator.samples}")
print(f"Testing data shape: {test_generator.samples}")


Found 1462 validated image filenames belonging to 5 classes.
Found 366 validated image filenames belonging to 5 classes.
Found 457 validated image filenames belonging to 5 classes.
Training data shape: 1462
Validation data shape: 366
Testing data shape: 457


## **Model 1: VGG16**

### **Initialisation**

In [28]:
# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all the base model layers to retain pre-trained weights initially
for layer in base_model.layers:
    layer.trainable = False

# Custom model on top of VGG16
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')  # Adjust the number of classes as per your dataset
])

# Compile the model
optimizer = Adam(learning_rate=0.0001)  # Small learning rate for fine-tuning
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 27562821 (105.14 MB)
Trainable params: 12848133 (49.01 MB)
Non-trainable params: 14714688 (56.13 MB)
______

### **Training**

In [29]:
# Callbacks for optimization
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model_vgg16.h5', monitor='val_loss', save_best_only=True)
]

# Class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks,
    class_weight=class_weight_dict  # Apply class weights
)

Epoch 1/30
46/46 [==============================] - ETA: 0s - loss: 1.6861 - accuracy: 0.2599

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


46/46 [==============================] - 75s 2s/step - loss: 1.6861 - accuracy: 0.2599 - val_loss: 1.5574 - val_accuracy: 0.2322
Epoch 2/30
46/46 [==============================] - 42s 909ms/step - loss: 1.4355 - accuracy: 0.3776 - val_loss: 1.3155 - val_accuracy: 0.4536
Epoch 3/30
46/46 [==============================] - 42s 917ms/step - loss: 1.3806 - accuracy: 0.3899 - val_loss: 1.3972 - val_accuracy: 0.2923
Epoch 4/30
46/46 [==============================] - 42s 902ms/step - loss: 1.3805 - accuracy: 0.3871 - val_loss: 1.3088 - val_accuracy: 0.4699
Epoch 5/30
46/46 [==============================] - 43s 906ms/step - loss: 1.3439 - accuracy: 0.4090 - val_loss: 1.2844 - val_accuracy: 0.4973
Epoch 6/30
46/46 [==============================] - 43s 885ms/step - loss: 1.3366 - accuracy: 0.4083 - val_loss: 1.2978 - val_accuracy: 0.4399
Epoch 7/30
46/46 [==============================] - 41s 895ms/step - loss: 1.3159 - accuracy: 0.4227 - val_loss: 1.2139 - val_accuracy: 0.5301
Epoch 8/30
46

In [ ]:
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

15/15 [==============================] - 8s 549ms/step - loss: 1.1567 - accuracy: 0.5252
Test Accuracy: 52.52%


## **Model 2: RESNET50**

### **Initialisation**

In [22]:
# Load the pre-trained ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model initially
for layer in base_model.layers:
    layer.trainable = False

# Define the custom model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Efficient pooling for ResNet50
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')  # Adjust the number of classes
])

# Compile the model
optimizer = Adam(learning_rate=0.0001)  # Use a small learning rate for transfer learning
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 24639365 (93.99 MB)
Trainable params: 1051653 (4.01 MB)
Non-trainable params: 23587712 (89.98 MB)
__________

### **Training**

In [ ]:
# Callbacks for optimization
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model_resnet50.h5', monitor='val_loss', save_best_only=True)
]

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks,
    class_weight=class_weight_dict  # Use class weights
)


Epoch 1/30
 4/46 [=>............................] - ETA: 3:05 - loss: 2.1800 - accuracy: 0.1406

In [1]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

NameError: name 'model' is not defined

## **Model 4: Custom Build Model 2:**

### **Initialisation**

In [26]:
# Define the custom CNN architecture
model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Second Convolutional Block
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    # Third Convolutional Block
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    # Flattening and Dense Layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    # Output Layer
    layers.Dense(5, activation='softmax')  # Adjust the number of classes
])

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 112, 112, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 128)     5

### **Training**

In [27]:
# Callbacks for optimization
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_custom_cnn.h5', monitor='val_loss', save_best_only=True)
]

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=callbacks,
    class_weight=class_weight_dict
)

Epoch 1/30
46/46 [==============================] - ETA: 0s - loss: 2.6075 - accuracy: 0.2579

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-490f74735c06>", line 16, in <cell line: 16>
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1856, in fit
    val_logs = self.evaluate(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2296, in evaluate
    logs = test_function_runner.run_step(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 4108, in run_step
    tmp_logs = self._function(dataset_or_iterator)
  

TypeError: object of type 'NoneType' has no len()

In [4]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

NameError: name 'model' is not defined

## **Model 5: Ensemble Model**